<a href="https://colab.research.google.com/github/rufous86/hh_api/blob/main/hh_api.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [145]:
import requests
from tqdm.notebook import tqdm
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('seaborn')
import plotly.express as px

In [146]:
res = []
def get_vacancy(vacancies, pages=50):
    for vacancy in vacancies:
        print(f'collecting {vacancy}')
        for page in tqdm(range(pages)):
            params = {
                'text': f'NAME:{vacancy}',
                'page': page,
                'per_page': 100
                }
            req = requests.get('https://api.hh.ru/vacancies', params).json()
            if 'items' in req.keys():
                res.extend(req['items'])

vacancies = ['аналитик', 'данные', 'data', 'analyst']
get_vacancy(vacancies)

data = pd.DataFrame(res)
data.shape

collecting аналитик


  0%|          | 0/50 [00:00<?, ?it/s]

collecting данные


  0%|          | 0/50 [00:00<?, ?it/s]

collecting data


  0%|          | 0/50 [00:00<?, ?it/s]

collecting analyst


  0%|          | 0/50 [00:00<?, ?it/s]

(5074, 30)

In [147]:
df = data.copy()
columns = ['name', 'salary', 'schedule', 'area', 'address']
df = df[columns]
df.head()

,name,salary,schedule,area,address
0,Аналитик,"{'from': 700000, 'to': 900000, 'currency': 'KZ...","{'id': 'fullDay', 'name': 'Полный день'}","{'id': '160', 'name': 'Алматы', 'url': 'https:...","{'city': 'Уральск', 'street': 'Туркестанская у..."
1,Аналитик,"{'from': 70000, 'to': 70000, 'currency': 'RUR'...","{'id': 'fullDay', 'name': 'Полный день'}","{'id': '26', 'name': 'Воронеж', 'url': 'https:...","{'city': 'Воронеж', 'street': 'улица Ворошилов..."
2,Системный аналитик\Бизнес аналитик,"{'from': 150000, 'to': 250000, 'currency': 'RU...","{'id': 'fullDay', 'name': 'Полный день'}","{'id': '2', 'name': 'Санкт-Петербург', 'url': ...","{'city': 'Санкт-Петербург', 'street': 'улица М..."
3,Бизнес-аналитик,"{'from': 130000, 'to': 130000, 'currency': 'RU...","{'id': 'fullDay', 'name': 'Полный день'}","{'id': '2', 'name': 'Санкт-Петербург', 'url': ...","{'city': 'Санкт-Петербург', 'street': 'Полтавс..."
4,Аналитик,"{'from': 200000, 'to': 200000, 'currency': 'KZ...","{'id': 'fullDay', 'name': 'Полный день'}","{'id': '160', 'name': 'Алматы', 'url': 'https:...","{'city': 'Алматы', 'street': 'улица Сатпаева',..."


In [148]:
df["city"] = (df["area"]
                     .apply(lambda x: x.get("name") 
                                      if isinstance(x, dict) 
                                      else np.nan))
df["schedule"] = (df["schedule"]
                     .apply(lambda x: x.get("id") 
                                      if isinstance(x, dict) 
                                      else np.nan))
df["salary_from"] = (df["salary"]
                     .apply(lambda x: x.get("from") 
                                      if isinstance(x, dict) 
                                      else np.nan))
df["salary_to"] = (df["salary"]
                     .apply(lambda x: x.get("to") 
                                      if isinstance(x, dict) 
                                      else np.nan))
df["salary_currency"] = (df["salary"]
                     .apply(lambda x: x.get("currency") 
                                      if isinstance(x, dict) 
                                      else np.nan))
df["lat"] = (df["address"]
                     .apply(lambda x: x.get("lat") 
                                      if isinstance(x, dict) 
                                      else np.nan))
df["lon"] = (df["address"]
                     .apply(lambda x: x.get("lng") 
                                      if isinstance(x, dict) 
                                      else np.nan))

In [149]:
df.head()

,name,salary,schedule,area,address,city,salary_from,salary_to,salary_currency,lat,lon
0,Аналитик,"{'from': 700000, 'to': 900000, 'currency': 'KZ...",fullDay,"{'id': '160', 'name': 'Алматы', 'url': 'https:...","{'city': 'Уральск', 'street': 'Туркестанская у...",Алматы,700000.0,900000.0,KZT,51.232035,51.455805
1,Аналитик,"{'from': 70000, 'to': 70000, 'currency': 'RUR'...",fullDay,"{'id': '26', 'name': 'Воронеж', 'url': 'https:...","{'city': 'Воронеж', 'street': 'улица Ворошилов...",Воронеж,70000.0,70000.0,RUR,51.653696,39.178566
2,Системный аналитик\Бизнес аналитик,"{'from': 150000, 'to': 250000, 'currency': 'RU...",fullDay,"{'id': '2', 'name': 'Санкт-Петербург', 'url': ...","{'city': 'Санкт-Петербург', 'street': 'улица М...",Санкт-Петербург,150000.0,250000.0,RUR,59.888803,30.279828
3,Бизнес-аналитик,"{'from': 130000, 'to': 130000, 'currency': 'RU...",fullDay,"{'id': '2', 'name': 'Санкт-Петербург', 'url': ...","{'city': 'Санкт-Петербург', 'street': 'Полтавс...",Санкт-Петербург,130000.0,130000.0,RUR,59.927116,30.367925
4,Аналитик,"{'from': 200000, 'to': 200000, 'currency': 'KZ...",fullDay,"{'id': '160', 'name': 'Алматы', 'url': 'https:...","{'city': 'Алматы', 'street': 'улица Сатпаева',...",Алматы,200000.0,200000.0,KZT,43.236861,76.911248


In [150]:
df.tail()

,name,salary,schedule,area,address,city,salary_from,salary_to,salary_currency,lat,lon
5069,Инженер-лаборант/Химик-эксперт/Химик-аналитик ...,"{'from': 40000, 'to': 60000, 'currency': 'RUR'...",fullDay,"{'id': '2084', 'name': 'Дзержинский', 'url': '...",None,Дзержинский,40000.0,60000.0,RUR,NaN,NaN
5070,Senior System analyst/ Системный аналитик (Obj...,None,fullDay,"{'id': '2', 'name': 'Санкт-Петербург', 'url': ...",None,Санкт-Петербург,NaN,NaN,NaN,NaN,NaN
5071,Стажер/Аналитик 1С,"{'from': 40000, 'to': 70000, 'currency': 'RUR'...",fullDay,"{'id': '1', 'name': 'Москва', 'url': 'https://...",{'city': 'Новомосковский административный окру...,Москва,40000.0,70000.0,RUR,55.637017,37.434954
5072,Аналитик данных (Риск-калькулятор),None,remote,"{'id': '1', 'name': 'Москва', 'url': 'https://...",None,Москва,NaN,NaN,NaN,NaN,NaN
5073,Аналитик отдела по работе с маркетплейсами,None,fullDay,"{'id': '2', 'name': 'Санкт-Петербург', 'url': ...",None,Санкт-Петербург,NaN,NaN,NaN,NaN,NaN


In [151]:
df['salary_currency'].value_counts()

RUR    1281
USD      46
KZT      34
UZS      12
EUR      12
BYR      10
KGS       1
AZN       1
Name: salary_currency, dtype: int64

In [152]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5074 entries, 0 to 5073
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   name             5074 non-null   object 
 1   salary           1397 non-null   object 
 2   schedule         5074 non-null   object 
 3   area             5074 non-null   object 
 4   address          2529 non-null   object 
 5   city             5074 non-null   object 
 6   salary_from      1199 non-null   float64
 7   salary_to        816 non-null    float64
 8   salary_currency  1397 non-null   object 
 9   lat              2382 non-null   float64
 10  lon              2382 non-null   float64
dtypes: float64(4), object(7)
memory usage: 436.2+ KB


In [153]:
df = df[df['salary_currency'] == 'RUR'][['name', 'schedule', 'city', 'salary_from', 'salary_to', 'salary_currency', 'lat', 'lon']]

In [154]:
df = df.dropna(axis=0, subset=['lat', 'lon'])
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 753 entries, 1 to 5071
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   name             753 non-null    object 
 1   schedule         753 non-null    object 
 2   city             753 non-null    object 
 3   salary_from      650 non-null    float64
 4   salary_to        426 non-null    float64
 5   salary_currency  753 non-null    object 
 6   lat              753 non-null    float64
 7   lon              753 non-null    float64
dtypes: float64(4), object(4)
memory usage: 52.9+ KB


In [155]:
df['salary_from'].min(), df['salary_from'].max()

(10000.0, 350000.0)

In [156]:
df['salary_to'].min(), df['salary_to'].max()

(160.0, 450000.0)

In [157]:
df[df['salary_to'] == 160]

,name,schedule,city,salary_from,salary_to,salary_currency,lat,lon
408,Системный аналитик,flexible,Москва,NaN,160.0,RUR,55.66076,37.54356


In [158]:
link = "https://raw.githubusercontent.com/timurkanaz/Russia_geojson_OSM/master/GeoJson's/Countries/Russia_regions.geojson"

In [159]:
from urllib.request import urlopen
import json
with urlopen(link) as response:
    counties = json.load(response)

# counties["features"][0]

In [164]:
px.scatter_geo?

In [166]:
fig = px.scatter_geo(df, lat='lat', lon='lon', text='city', width=1600, height=1600)
fig.show()

In [162]:
cities = pd.DataFrame()

cities['total_vac'] = (df
 .groupby('city')['name']
 .agg('count')
 )

cities['lat'] = (df
 .groupby('city')
 .agg({'lat': 'mean'})
 )

cities['lon'] = (df
 .groupby('city')
 .agg({'lon': 'mean'})
 )

cities['city'] = cities.index

cities.head()

,total_vac,lat,lon,city
city,,,,
Архангельск,1,64.552702,40.520388,Архангельск
Барнаул,6,53.346661,83.729292,Барнаул
Белгород,4,50.597674,36.587261,Белгород
Бийск,1,52.548723,85.234319,Бийск
Благовещенск (Амурская область),2,50.267470,127.538681,Благовещенск (Амурская область)


In [167]:
fig = px.scatter_geo(cities, lat='lat', lon='lon', text='city', color='total_vac', size='total_vac', width=1600, height=1600)
fig.show()